# Week 1 - January 23

## **Monday 01/02**

### WSL and X-session

I have left on-going tests last week of 22.

Trying to start a X-session on ubuntu 22.04 within this new version of WSL.


#### **systemctl-sysusers failed [FIXED]**

seen in `journalctl -r` when starting `gnome-session` some errors linked to `systemd-sysusers.service`.

And I can see failed services with

```bash
sudo systemctl --failed
  UNIT                     LOAD   ACTIVE SUB    DESCRIPTION
● systemd-sysusers.service loaded failed failed Create System Users

LOAD   = Reflects whether the unit definition was properly loaded.
ACTIVE = The high-level unit activation state, i.e. generalization of SUB.
SUB    = The low-level unit activation state, values depend on unit type.
1 loaded units listed.
```

by getting additional info

```bash
sudo systemctl status  systemd-sysusers
× systemd-sysusers.service - Create System Users
     Loaded: loaded (/lib/systemd/system/systemd-sysusers.service; static)
     Active: failed (Result: exit-code) since Mon 2023-01-02 09:44:29 CET; 10min ago
       Docs: man:sysusers.d(5)
             man:systemd-sysusers.service(8)
   Main PID: 1803 (code=exited, status=243/CREDENTIALS)

Jan 02 09:44:29 L001LPF3RKAR5 systemd[1]: Starting Create System Users...
Jan 02 09:44:29 L001LPF3RKAR5 systemd[1803]: systemd-sysusers.service: Failed to set up credentials: Protocol error
Jan 02 09:44:29 L001LPF3RKAR5 systemd[1803]: systemd-sysusers.service: Failed at step CREDENTIALS spawning systemd-sysusers: Protocol error
Jan 02 09:44:29 L001LPF3RKAR5 systemd[1]: systemd-sysusers.service: Main process exited, code=exited, status=243/CREDENTIALS
Jan 02 09:44:29 L001LPF3RKAR5 systemd[1]: systemd-sysusers.service: Failed with result 'exit-code'.
Jan 02 09:44:29 L001LPF3RKAR5 systemd[1]: Failed to start Create System Users.
```

user encountered this issue: https://anthony-f-tannous.medium.com/run-systemd-on-wsl2-distro-18a9775cb2f2

and suggests a workaround:

```bash
sudo systemctl edit systemd-sysusers.service

#here is the content to be added:

### Anything between here and the comment below will become the new contents of the file[Service]
[Service]
LoadCredential=
### Lines below this comment will be discarded### /lib/systemd/system/systemd-sysusers.service

#check this is active
cat /etc/systemd/system/systemd-sysusers.service.d/override.conf

#restart service
sudo systemctl restart systemd-sysusers.service
```

#### **gnome-session failed**

```bash
sudo journalctl -r
gnome-session-binary[3048]: Entering running state
gnome-session[3048]: _IceTransmkdir: ERROR: euid != 0,directory /tmp/.ICE-unix will not be created.
gnome-session-binary[3048]: WARNING: Could not get session id for session. Check that logind is properly installed and pam_systemd is getting used at login.
gnome-session[3048]: gnome-session-binary[3048]: WARNING: Could not get session id for session. Check that logind is properly installed and pam_systemd is getting used at login.
gnome-session-binary[3048]: WARNING: Falling back to non-systemd startup procedure due to error: GDBus.Error:org.freedesktop.DBus.Error.InvalidArgs: Unit name gnome-session-(null)@gnome.target is not valid.
gnome-session[3048]: gnome-session-binary[3048]: WARNING: Falling back to non-systemd startup procedure due to error: GDBus.Error:org.freedesktop.DBus.Error.InvalidArgs: Unit name gnome-session-(null)@gnome.target is not valid.
```

to try https://askubuntu.com/questions/1442663/cannot-start-gnome-session-after-enabling-systemd-on-latest-wsl2

Activate rc.local service: (see https://www.cyberciti.biz/faq/how-to-enable-rc-local-shell-script-on-systemd-while-booting-linux-system/)

```bash
sudo systemctl is-enabled rc-local.service
(if static it means it has be activated)
sudo systemctl enable rc-local.service

#create rc.local file
sudo vi /etc/rc.local

#!/bin/sh -e#!/bin/sh -e
mount -o rw,remount /tmp/.X11-unix/
chmod +t /tmp/.X11-unix
chmod o+rw /dev/dri/renderD128
/usr/libexec/at-spi-bus-launcher --launch-immediately &
exit 0

#save & exit
sudo chmod 700 /etc/rc.local
```

create `/etc/pam.d/system/systemd-user` with this content:


```text
# This file is part of systemd.
#
# Used by systemd --user instances.

@include common-account

session  required pam_loginuid.so
session  required pam_limits.so
@include common-session-noninteractive
session optional pam_systemd.so
```

and restart wsl

#### **from scratch**


from powershell

```bash
wsl --import ubuntu-x11 C:\wsl\Ubuntu-22.04_X11\instance C:\wsl\Ubuntu-22.04_X11\download\ubuntu-jammy-wsl-amd64-wsl.rootfs.tar.gz
wsl -d ubuntu-x11
cd
cp /mnt/c/wsl/Ubuntu-22.04_X11/download/setup_wsl_* ~/
chmod +x setup_wsl_root.sh
./setup_wsl_root.sh
```

from powershell
```bash
wsl -t ubuntu-x11
```

start ubuntu-x11 from windows terminal menu

and enter informations requested (ssh keys (paste to gitlab), update certificates (enter password)). Other steps are automatic.

Other steps:
- activate systemd (by modifying /etc/wsl.conf)
- restart


Here we have a fully working distro. And we will test what is suggested at : https://askubuntu.com/a/1445415

* Create the file /etc/pam.d/system/systemd-user
* Restart distro
* `sudo apt install ubuntu-desktop acpi-support-`
* prevent gdm service to start
    - `sudo systemctl stop gdm.service`
    - `sudo systemctl disable gdm.service`
* activate x11 config at boot:
    - ```bash
        sudo systemctl is-enabled rc-local.service
        (if static it means it has be activated)
        sudo systemctl enable rc-local.service
        ```
    - ```bash
        #create rc.local file
        sudo vi /etc/rc.local

        #!/bin/sh -e
        mount -o rw,remount /tmp/.X11-unix/
        chmod +t /tmp/.X11-unix
        chmod o+rw /dev/dri/renderD128
        /usr/libexec/at-spi-bus-launcher --launch-immediately &
        exit 0

        #save & exit
        sudo chmod 700 /etc/rc.local
        ```
* `sudo usermod -a -G adm guillaume`
* Restart distro
* `XDG_SESSION_TYPE=x11 gnome-session --disable-acceleration-check --session=ubuntu --systemd-service > /dev/null 2>&1`
